In [1]:
import pandas as pd
import numpy as np
from scipy.stats import beta
import math
import random
import copy
from itertools import chain

In [2]:
import os

os.getcwd()

'/Users/jasonch/mycode/school/Beta'

# Data

In [3]:
df = pd.read_csv('./Data/movielen_100k.csv', header = 0, index_col = 'movie_id')
df = df.fillna(0)
#df.head()
df.tail()

,r1,r2,r3,r4,r5
movie_id,,,,,
1678,1.0,0.0,0.0,0.0,0.0
1679,0.0,0.0,1.0,0.0,0.0
1680,0.0,1.0,0.0,0.0,0.0
1681,0.0,0.0,1.0,0.0,0.0
1682,0.0,0.0,1.0,0.0,0.0


In [4]:
df['source_round_avgr'] = round((df['r1'] * 1 + df['r2'] * 2 + df['r3'] * 3 + df['r4'] * 4 + df['r5'] * 5) / (df['r1'] + df['r2'] + df['r3'] + df['r4'] + df['r5']))
df.head()
#df.tail()

,r1,r2,r3,r4,r5,source_round_avgr
movie_id,,,,,,
1,8.0,27.0,96.0,202.0,119.0,4.0
2,8.0,17.0,55.0,42.0,9.0,3.0
3,11.0,20.0,25.0,23.0,11.0,3.0
4,6.0,24.0,57.0,93.0,29.0,4.0
5,4.0,11.0,32.0,33.0,6.0,3.0


In [5]:
len(df)

1682

# Define Function

In [5]:
# attacked item
def atted(numb, totalitems):
    atteditem = []

    for i in range(numb):
        atteditem.append(random.choice([x for x in range(1, totalitems + 1) if x not in atteditem]))
    
    return atteditem

In [6]:
# fill items
def filler(fill_size, fa, totalitems, atteditem):
    fillitem = []
        
    for j in range(fill_size * fa):
        fillitem.append(random.choice([x for x in range(1, totalitems + 1) if x not in chain(atteditem, fillitem)]))
        
    return fillitem

In [7]:
# inject data
def filldata(df, atteditem, fillitem):
    for m in atteditem:
        df.loc[m, 'r5'] += 1    # push
        #df.loc[m, 'r1'] += 1     # nuke
    
    for n in fillitem:
        if df.loc[n, 'source_round_avgr'] == 5:
            df.loc[n, 'r5'] += 1
        elif df.loc[n, 'source_round_avgr'] == 4:
            df.loc[n, 'r4'] += 1
        elif df.loc[n, 'source_round_avgr'] == 3:
            df.loc[n, 'r3'] += 1
        elif df.loc[n, 'source_round_avgr'] == 2:
            df.loc[n, 'r2'] += 1
        else:
            df.loc[n, 'r1'] += 1
    
    return df

In [8]:
def calsum(df):
    df['sum'] = df['r1'] + df['r2'] + df['r3'] + df['r4'] + df['r5']
    df['avgr'] = (df['r1'] * 1 + df['r2'] * 2 + df['r3'] * 3 + df['r4'] * 4 + df['r5'] * 5) / (df['r1'] + df['r2'] + df['r3'] + df['r4'] + df['r5'])
    df['round_avgr'] = round(df['avgr'])
    df['std'] =  (((df['r1'] * ((1 - df['avgr']) ** 2)) + (df['r2'] * ((2 - df['avgr']) ** 2)) + (df['r3'] * ((3 - df['avgr']) ** 2)) + (df['r4'] * ((4 - df['avgr']) ** 2)) + (df['r5'] * ((5 - df['avgr']) ** 2))) / df['sum']) ** 0.5
    df['std'] = df['std'].fillna(0)

    df.loc['every_col_sum'] = df.apply(lambda x: x.sum())
    
    df.at['every_col_sum', 'avgr'] = (df.at['every_col_sum', 'r1'] * 1 + df.at['every_col_sum', 'r2'] * 2 + df.at['every_col_sum', 'r3'] * 3 + df.at['every_col_sum', 'r4'] * 4 + df.at['every_col_sum', 'r5'] * 5) / (df.at['every_col_sum', 'r1'] + df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'])
    df.at['every_col_sum', 'round_avgr'] = round(df.at['every_col_sum', 'avgr'])
    df.at['every_col_sum', 'std'] =  (((df.at['every_col_sum', 'r1'] * ((1 - df.at['every_col_sum', 'avgr']) ** 2)) + (df.at['every_col_sum', 'r2'] * ((2 - df.at['every_col_sum', 'avgr']) ** 2)) + (df.at['every_col_sum', 'r3'] * ((3 - df.at['every_col_sum', 'avgr']) ** 2)) + (df.at['every_col_sum', 'r4'] * ((4 - df.at['every_col_sum', 'avgr']) ** 2)) + (df.at['every_col_sum', 'r5'] * ((5 - df.at['every_col_sum', 'avgr']) ** 2))) / (df.at['every_col_sum', 'r1'] + df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'])) ** 0.5    
    
    return df

In [9]:
# Five Methods
def five_method(df, quant):
    
    df_five = pd.DataFrame(columns = ['LB1', 'UB1', 'E1', 'LB2', 'UB2', 'E2', 'LB3', 'UB3', 'E3', 'LB4', 'UB4', 'E4', 'LB5', 'UB5', 'E5'], index = df.index)
    
    #[1,2,3,4,5]
    df_five['LB1'] = beta.ppf(quant, df['r3'] + df['r4'] + df['r5'] + 1, df['r1'] + df['r2'] + df['r3'] + 1)
    df_five['UB1'] = beta.ppf(1 - quant, df['r3'] + df['r4'] + df['r5']  + 1, df['r1'] + df['r2'] + df['r3'] + 1)
    df_five['E1'] = (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'] + 1) / (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'] + df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r1'] + 2) 

    #[3, 4, 5]
    df_five['LB2'] = beta.ppf(quant, df['r4'] + df['r5'] + 1, df['r3'] + df['r4']+ 1)
    df_five['UB2'] = beta.ppf(1 - quant, df['r4'] + df['r5'] + 1, df['r3'] + df['r4'] + 1)
    df_five['E2'] = (df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'] + 1) / (df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r5'] + df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + 2) 

    #[1, 2, 3,]
    df_five['LB3'] = beta.ppf(quant, df['r2'] + df['r3'] + 1, df['r1'] + df['r2'] + 1)
    df_five['UB3'] = beta.ppf(1 - quant, df['r2'] + df['r3'] + 1, df['r1'] + df['r2'] + 1)
    df_five['E3'] = (df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r3'] + 1) / (df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r1'] + df.at['every_col_sum', 'r2'] + 2) 

    #[2, 3, 4]
    df_five['LB4'] = beta.ppf(quant, df['r3'] + df['r4'] + 1, df['r2'] + df['r3'] + 1)
    df_five['UB4'] = beta.ppf(1 - quant, df['r3'] + df['r4'] + 1, df['r2'] + df['r3'] + 1)
    df_five['E4'] = (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + 1) / (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r4'] + df.at['every_col_sum', 'r2'] + df.at['every_col_sum', 'r3'] + 2) 

    #[1, 3, 5]
    df_five['LB5'] = beta.ppf(quant, df['r3'] + df['r5'] + 1, df['r1'] + df['r3'] + 1)
    df_five['UB5'] = beta.ppf(1 - quant, df['r3'] + df['r5'] + 1, df['r1'] + df['r3'] + 1)
    df_five['E5'] = (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r5'] + 1) / (df.at['every_col_sum', 'r3'] + df.at['every_col_sum', 'r5'] + df.at['every_col_sum', 'r1'] + df.at['every_col_sum', 'r3'] + 2) 
  
    df_five['ct'] = 0
    
    df_five = df_five[:-1]
    
    return df_five

In [10]:
# Detect
def detect_att(df, totalitems):
    for u in range(1, totalitems + 1):
        if (df.loc[u, 'E1'] < df.loc[u, 'LB1']) | (df.loc[u, 'E1'] > df.loc[u, 'UB1']):
            df.loc[u, 'ct'] += 1

        if (df.loc[u, 'E2'] < df.loc[u, 'LB2']) | (df.loc[u, 'E2'] > df.loc[u, 'UB2']):
            df.loc[u, 'ct'] += 1
    
        if (df.loc[u, 'E3'] < df.loc[u, 'LB3']) | (df.loc[u, 'E3'] > df.loc[u, 'UB3']):
            df.loc[u, 'ct'] += 1
    
        if (df.loc[u, 'E4'] < df.loc[u, 'LB4']) | (df.loc[u, 'E4'] > df.loc[u, 'UB4']):
            df.loc[u, 'ct'] += 1
    
        if (df.loc[u, 'E5'] < df.loc[u, 'LB5']) | (df.loc[u, 'E5'] > df.loc[u, 'UB5']):
            df.loc[u, 'ct'] += 1  

    return df

# Run

In [14]:
users = 943
totalitems = 1682

#for attack size
ats = round(users * 0.01)

#for fill size & attack item
fa = round(totalitems * 0.01)

beta_threshold = 3

In [ ]:
print('beta_threshold = ' + str(beta_threshold))
print('Programs Start....')

quants = [0.003]
numbs = [1]
attack_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
fill_sizes = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


for quant in quants:
    for numb in numbs:
        for attack_size in attack_sizes:
            for fill_size in fill_sizes:
                detrlist = []
                falslist = []
                accylist = []
                perlist = []       
                recalllist = []         
                fscorelist = []
    
                xbdetrlist = []
                xbfalslist = []
                xbaccylist = []
                xbperlist = []
                xbrecalllist = []  
                xbfscorelist = [] 
        
                for y in range(10):
                    #初始值
                    dfused = pd.DataFrame()
                    atteditem = []
                    allfill = []
                    dfsum = pd.DataFrame()
                    xbar_df = pd.DataFrame()
                    dfok = pd.DataFrame()
                    dffive = pd.DataFrame()
                    beta_count = pd.DataFrame()
                    xbard = pd.DataFrame()
                    xb_count = pd.DataFrame()

        
                    # Run
                    dfused = copy.deepcopy(df)
                    atteditems = atted(numb, totalitems)
    
                    k = 0
                    for k in range(attack_size * ats):
                        fillitems = []
                        fillitems = filler(fill_size, fa, totalitems, atteditems)
                        dfused = filldata(dfused, atteditems, fillitems)

                        #allfill.append(fillitems) 
    

                    dfsum = calsum(dfused)
                    xbar_df = copy.deepcopy(dfsum)

                    dffive = five_method(dfsum, quant)
                    beta_count = detect_att(dffive, totalitems)

                    xbard, Uxbar, Lxbar = xbar(xbar_df, totalitems)
                    xb_count = xbar_detect_att(xbard, Uxbar, Lxbar)

                    
                    ## beta ##
                    #detect rate
                    detr = len(beta_count.loc[atteditems][(beta_count.loc[atteditems, 'ct'] >= beta_threshold) == True]) / len(atteditems)
                    detrlist.append(detr)
          
                    #false rate
                    ffliter = []
                    for y in beta_count[(beta_count['ct'] >= beta_threshold) == True].index:
                        if y not in beta_count.loc[atteditems].index:
                            ffliter.append(y) 

                    fals = len(ffliter) / totalitems
                    falslist.append(fals)
        
    
                    ## write to csv ##
                    cont = {'Quanti' : quant, 'Attack Item' : str(numb) + 'items', 'Attack Size %' : attack_size, 'Fill Size %' : fill_size, 'Detect' : [len(beta_count[(beta_count['ct'] >= beta_threshold) == True])], 'Hit True #':len(beta_count.loc[atteditems][(beta_count.loc[atteditems]['ct'] >= beta_threshold) == True]), 'beta' : 'beta', 'Detec Rate %' : round(detr * 100, 2), 'False Rate %' : round(fals * 100, 2)}
                    contet = pd.DataFrame(cont)
    
                    if not os.path.isfile('./result/cont.csv'):
                        contet.to_csv('./result/cont.csv', index = False)
                    else:
                        contet.to_csv('./result/cont.csv', mode = 'a', header = False, index = False)
    
    
    
                 ## write to csv ##
                dff = {'beta_threshold' : beta_threshold, 'Quanti' : quant, 'Attack Item' : str(numb) + 'items', 'Attack Size %' : attack_size, 'Fill Size %' : fill_size, 'beta' : 'beta', 'Detect Rate %' : round(np.mean(detrlist) * 100, 2), 'False Rate %' : round(np.mean(falslist) * 100, 2)}
                result = pd.DataFrame(dff, index = [0])
    
                if not os.path.isfile('./result/result.csv'):
                    result.to_csv('./result/result.csv', index = False)
                else:   
                    result.to_csv('./result/result.csv', mode = 'a', header = False, index = False)


print('to csv is ok!!')